# 실습 3: Vector Database 구성

## 목표
- S3 Vector Bucket 생성 (소방서 + CCTV)
- 소방서 데이터 임베딩
- CCTV 데이터 임베딩
- Vector Search 테스트

## 3-1. 설정 로드

In [ ]:
from workshop_config import REGION, VECTOR_BUCKET_NAME, VECTOR_INDEX_NAME, CCTV_BUCKET_NAME, CCTV_INDEX_NAME
import boto3
import json
import pandas as pd

print(f"🌍 Region: {REGION}")
print(f"🗄️ 소방서 Bucket: {VECTOR_BUCKET_NAME}")
print(f"📊 소방서 Index: {VECTOR_INDEX_NAME}")
print(f"🗄️ CCTV Bucket: {CCTV_BUCKET_NAME}")
print(f"📊 CCTV Index: {CCTV_INDEX_NAME}")

---
## Part 1: 소방서 Vector Database
전국의 소방서 정보를 임베딩하여 벡터화합니다.

## 3-2. 소방서 S3 Vector Bucket 생성

In [ ]:
s3vectors = boto3.client('s3vectors', region_name=REGION)

try:
    response = s3vectors.create_vector_bucket(vectorBucketName=VECTOR_BUCKET_NAME)
    print(f"✅ Vector Bucket 생성: {VECTOR_BUCKET_NAME}")
except Exception as e:
    if 'AlreadyExists' in str(e) or 'AlreadyOwned' in str(e):
        print(f"ℹ️ Vector Bucket 이미 존재: {VECTOR_BUCKET_NAME}")
    else:
        raise e

## 3-3. 소방서 Vector Index 생성

In [ ]:
try:
    response = s3vectors.create_index(
        vectorBucketName=VECTOR_BUCKET_NAME,
        indexName=VECTOR_INDEX_NAME,
        dataType='float32',
        dimension=1024,
        distanceMetric='cosine'
    )
    print(f"✅ Vector Index 생성: {VECTOR_INDEX_NAME}")
except Exception as e:
    if 'AlreadyExists' in str(e):
        print(f"ℹ️ Index 이미 존재: {VECTOR_INDEX_NAME}")
    else:
        raise e

## 3-4. 소방서 데이터를 확인

csv에 저장되어있는 데이터를 조회해봅니다.

In [ ]:
import os

csv_path = os.path.join(os.getcwd(), '..', 'misc', 'firestation_20240901.csv')

if not os.path.exists(csv_path):
    print(f"⚠️ 파일 없음: {csv_path}")
else:
    df_fire = pd.read_csv(csv_path, encoding='euc-kr')
    print(f"✅ {len(df_fire)}개 소방서 데이터 로드")
    display(df_fire.head())

## 3-5. 소방서 데이터 임베딩

소방서 정보를 amazon titan 모델을 이용해 임베딩합니다.

In [ ]:
bedrock = boto3.client("bedrock-runtime", region_name=REGION)

print(f"🚀 소방서 임베딩 시작 (예상: {len(df_fire) * 0.5 / 60:.1f}분)\n")

vectors = []
batch_size = 50

for idx, row in df_fire.iterrows():
    lat, lon = float(row['X좌표']), float(row['Y좌표'])
    text = f"{row['소방서 및 안전센터명']} , 위치: {row['주소']} , 소속: {row['상위 본부명']} , 위도: {lat:.6f} , 경도: {lon:.6f}"
    
    if (idx + 1) % 100 == 0:
        print(f"처리중 [{idx+1}/{len(df_fire)}]")
    
    response = bedrock.invoke_model(
        modelId="amazon.titan-embed-text-v2:0",
        body=json.dumps({"inputText": text, "dimensions": 1024, "normalize": True})
    )
    embedding = json.loads(response["body"].read())["embedding"]
    
    vectors.append({
        "key": f"firestation_{idx}",
        "data": {"float32": embedding},
        "metadata": {
            "name": str(row['소방서 및 안전센터명']),
            "address": str(row['주소']),
            "phone": str(row['전화번호']),
            "latitude": str(lat),
            "longitude": str(lon),
            "type": str(row['유형']),
            "headquarters": str(row['상위 본부명'])
        }
    })
    
    if len(vectors) == batch_size:
        s3vectors.put_vectors(vectorBucketName=VECTOR_BUCKET_NAME, indexName=VECTOR_INDEX_NAME, vectors=vectors)
        vectors = []

if vectors:
    s3vectors.put_vectors(vectorBucketName=VECTOR_BUCKET_NAME, indexName=VECTOR_INDEX_NAME, vectors=vectors)

print(f"\n🎉 소방서 임베딩 완료!")

## 3-6. 소방서 Vector Search 테스트

In [ ]:
test_address = "서울특별시 서초구 방배중앙로 06681"
print(f"🔍 테스트: {test_address}\n")

# 임베딩
resp = bedrock.invoke_model(
    modelId="amazon.titan-embed-text-v2:0",
    body=json.dumps({"inputText": test_address, "dimensions": 1024, "normalize": True})
)
query_embedding = json.loads(resp["body"].read())["embedding"]

# 검색
results = s3vectors.query_vectors(
    vectorBucketName=VECTOR_BUCKET_NAME,
    indexName=VECTOR_INDEX_NAME,
    queryVector={"float32": query_embedding},
    topK=5,
    returnMetadata=True
)

print("✅ 가까운 소방서 5곳:\n")
for i, item in enumerate(results["vectors"], 1):
    m = item.get("metadata", {})
    print(f"{i}. {m.get('name')} - {m.get('address')}")

---
## Part 2: CCTV Vector Database

천안시 CCTV 정보를 임베딩하여 벡터화합니다.

## 3-7. CCTV S3 Vector Bucket 생성

In [ ]:
try:
    response = s3vectors.create_vector_bucket(vectorBucketName=CCTV_BUCKET_NAME)
    print(f"✅ CCTV Vector Bucket 생성: {CCTV_BUCKET_NAME}")
except Exception as e:
    if 'AlreadyExists' in str(e) or 'AlreadyOwned' in str(e):
        print(f"ℹ️ CCTV Vector Bucket 이미 존재: {CCTV_BUCKET_NAME}")
    else:
        raise e

## 3-8. CCTV Vector Index 생성

In [ ]:
try:
    response = s3vectors.create_index(
        vectorBucketName=CCTV_BUCKET_NAME,
        indexName=CCTV_INDEX_NAME,
        dataType='float32',
        dimension=1024,
        distanceMetric='cosine'
    )
    print(f"✅ CCTV Vector Index 생성: {CCTV_INDEX_NAME}")
except Exception as e:
    if 'AlreadyExists' in str(e):
        print(f"ℹ️ CCTV Index 이미 존재: {CCTV_INDEX_NAME}")
    else:
        raise e

## 3-9. CCTV 데이터 로드

In [ ]:
cctv_csv_path = os.path.join(os.getcwd(), '..', 'misc', 'cheonan_cctv.csv')

if not os.path.exists(cctv_csv_path):
    print(f"⚠️ 파일 없음: {cctv_csv_path}")
else:
    df_cctv = pd.read_csv(cctv_csv_path, encoding='utf-8')
    print(f"✅ {len(df_cctv)}개 CCTV 데이터 로드")
    display(df_cctv.head())

## 3-10. CCTV 데이터 임베딩

In [ ]:
print(f"🚀 CCTV 임베딩 시작 (예상: {len(df_cctv) * 0.5 / 60:.1f}분)\n")

vectors = []
batch_size = 50

for idx, row in df_cctv.iterrows():
    longitude = float(row['경도'])
    latitude = float(row['위도'])
    
    # CCTV 정보를 텍스트로 변환
    text = f"{row['설치위치명']}, 위치: {row['설치위치주소']}, 위도: {latitude:.6f}, 경도: {longitude:.6f}"
    
    if (idx + 1) % 100 == 0:
        print(f"처리중 [{idx+1}/{len(df_cctv)}]")
    
    response = bedrock.invoke_model(
        modelId="amazon.titan-embed-text-v2:0",
        body=json.dumps({"inputText": text, "dimensions": 1024, "normalize": True})
    )
    embedding = json.loads(response["body"].read())["embedding"]
    
    vectors.append({
        "key": f"cctv_{idx}",
        "data": {"float32": embedding},
        "metadata": {
            "cctv_id": str(row['CCTV관리번호']),
            "name": str(row['설치위치명']),
            "address": str(row['설치위치주소']),
            "latitude": str(latitude),
            "longitude": str(longitude),
            "stream_url": str(row['스트리밍 프로토콜(HTTP)주소'])
        }
    })
    
    if len(vectors) == batch_size:
        s3vectors.put_vectors(vectorBucketName=CCTV_BUCKET_NAME, indexName=CCTV_INDEX_NAME, vectors=vectors)
        vectors = []

if vectors:
    s3vectors.put_vectors(vectorBucketName=CCTV_BUCKET_NAME, indexName=CCTV_INDEX_NAME, vectors=vectors)

print(f"\n🎉 CCTV 임베딩 완료!")

## 3-11. CCTV Vector Search 테스트

In [ ]:
test_cctv_address = "충청남도 천안시 서북구 신당동"
print(f"🔍 CCTV 테스트: {test_cctv_address}\n")

# 임베딩
resp = bedrock.invoke_model(
    modelId="amazon.titan-embed-text-v2:0",
    body=json.dumps({"inputText": test_cctv_address, "dimensions": 1024, "normalize": True})
)
query_embedding = json.loads(resp["body"].read())["embedding"]

# 검색
results = s3vectors.query_vectors(
    vectorBucketName=CCTV_BUCKET_NAME,
    indexName=CCTV_INDEX_NAME,
    queryVector={"float32": query_embedding},
    topK=3,
    returnMetadata=True
)

print("✅ 가까운 CCTV 3곳:\n")
for i, item in enumerate(results["vectors"], 1):
    m = item.get("metadata", {})
    print(f"{i}. {m.get('name')} - {m.get('address')}")
    print(f"   스트리밍: {m.get('stream_url')}\n")

## ✅ 실습 3 완료!

### 생성된 리소스
- S3 Vector Bucket: `firestation-location-xy`
- Vector Index: `fire-station`
- 임베딩된 소방서 데이터
- S3 Vector Bucket: `cctv-m3u8`
- Vector Index: `cctv-cheonan`
- 임베딩된 CCTV 데이터

👉 다음: **lab4_create_agent.ipynb**